## Embeddings

Dalam contoh sebelumnya, kita bekerja dengan vektor bag-of-words berdimensi tinggi dengan panjang `vocab_size`, dan kita secara eksplisit mengonversi dari vektor representasi posisi berdimensi rendah ke representasi one-hot yang jarang. Representasi one-hot ini tidak efisien dalam penggunaan memori, selain itu, setiap kata diperlakukan secara independen satu sama lain, yaitu vektor one-hot encoded tidak menunjukkan kesamaan semantik antara kata-kata.

Dalam unit ini, kita akan melanjutkan eksplorasi dataset **News AG**. Untuk memulai, mari kita muat data dan mengambil beberapa definisi dari notebook sebelumnya.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Apa itu embedding?

Ide **embedding** adalah merepresentasikan kata-kata dengan vektor berdimensi rendah yang padat, yang mencerminkan makna semantik dari sebuah kata. Kita akan membahas cara membangun embedding kata yang bermakna nanti, tetapi untuk saat ini, anggap saja embedding sebagai cara untuk menurunkan dimensi vektor kata.

Jadi, lapisan embedding akan menerima sebuah kata sebagai input, dan menghasilkan vektor output dengan ukuran `embedding_size` yang ditentukan. Dalam beberapa hal, ini sangat mirip dengan lapisan `Linear`, tetapi alih-alih menerima vektor one-hot encoded, lapisan ini dapat menerima nomor kata sebagai input.

Dengan menggunakan lapisan embedding sebagai lapisan pertama dalam jaringan kita, kita dapat beralih dari model bag-of-words ke model **embedding bag**, di mana kita pertama-tama mengonversi setiap kata dalam teks kita menjadi embedding yang sesuai, lalu menghitung beberapa fungsi agregat atas semua embedding tersebut, seperti `sum`, `average`, atau `max`.

![Gambar yang menunjukkan pengklasifikasi embedding untuk lima kata dalam urutan.](../../../../../translated_images/id/embedding-classifier-example.b77f021a7ee67eee.webp)

Jaringan saraf pengklasifikasi kita akan dimulai dengan lapisan embedding, kemudian lapisan agregasi, dan pengklasifikasi linear di atasnya:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Mengatasi Ukuran Urutan Variabel

Sebagai hasil dari arsitektur ini, minibatch untuk jaringan kita perlu dibuat dengan cara tertentu. Pada unit sebelumnya, saat menggunakan bag-of-words, semua tensor BoW dalam sebuah minibatch memiliki ukuran yang sama `vocab_size`, terlepas dari panjang sebenarnya dari urutan teks kita. Setelah kita beralih ke word embeddings, kita akan memiliki jumlah kata yang bervariasi di setiap sampel teks, dan saat menggabungkan sampel-sampel tersebut ke dalam minibatch, kita harus menerapkan padding.

Hal ini dapat dilakukan dengan menggunakan teknik yang sama, yaitu menyediakan fungsi `collate_fn` ke sumber data:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Melatih pengklasifikasi embedding

Sekarang setelah kita mendefinisikan dataloader yang sesuai, kita dapat melatih model menggunakan fungsi pelatihan yang telah kita definisikan di unit sebelumnya:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Catatan**: Kami hanya melatih untuk 25k catatan di sini (kurang dari satu epoch penuh) demi menghemat waktu, tetapi Anda dapat melanjutkan pelatihan, menulis fungsi untuk melatih selama beberapa epoch, dan bereksperimen dengan parameter tingkat pembelajaran untuk mencapai akurasi yang lebih tinggi. Anda seharusnya dapat mencapai akurasi sekitar 90%.


### Lapisan EmbeddingBag dan Representasi Urutan dengan Panjang Variabel

Dalam arsitektur sebelumnya, kita perlu menambahkan padding pada semua urutan agar memiliki panjang yang sama untuk dimasukkan ke dalam minibatch. Ini bukan cara yang paling efisien untuk merepresentasikan urutan dengan panjang variabel - pendekatan lain adalah menggunakan vektor **offset**, yang akan menyimpan offset dari semua urutan yang disimpan dalam satu vektor besar.

![Gambar yang menunjukkan representasi urutan dengan offset](../../../../../translated_images/id/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Note**: Pada gambar di atas, kami menunjukkan urutan karakter, tetapi dalam contoh ini kami bekerja dengan urutan kata. Namun, prinsip umum merepresentasikan urutan dengan vektor offset tetap sama.

Untuk bekerja dengan representasi offset, kita menggunakan lapisan [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). Lapisan ini mirip dengan `Embedding`, tetapi menerima vektor konten dan vektor offset sebagai input, dan juga mencakup lapisan penghitungan rata-rata, yang bisa berupa `mean`, `sum`, atau `max`.

Berikut adalah jaringan yang dimodifikasi yang menggunakan `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Untuk mempersiapkan dataset untuk pelatihan, kita perlu menyediakan fungsi konversi yang akan mempersiapkan vektor offset:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Perhatikan bahwa, tidak seperti pada semua contoh sebelumnya, jaringan kita sekarang menerima dua parameter: vektor data dan vektor offset, yang memiliki ukuran berbeda. Demikian pula, pemuat data kita juga memberikan 3 nilai kepada kita, bukan 2: baik vektor teks maupun vektor offset disediakan sebagai fitur. Oleh karena itu, kita perlu sedikit menyesuaikan fungsi pelatihan kita untuk mengatasi hal tersebut:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantic Embeddings: Word2Vec

Dalam contoh sebelumnya, lapisan embedding model belajar memetakan kata-kata ke representasi vektor, namun representasi tersebut tidak memiliki banyak makna semantis. Akan lebih baik jika kita dapat mempelajari representasi vektor di mana kata-kata yang serupa atau sinonim memiliki vektor yang dekat satu sama lain berdasarkan jarak vektor tertentu (misalnya, jarak euclidean).

Untuk mencapai hal tersebut, kita perlu melatih model embedding kita terlebih dahulu pada kumpulan teks yang besar dengan cara tertentu. Salah satu metode pertama untuk melatih embedding semantis disebut [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Metode ini didasarkan pada dua arsitektur utama yang digunakan untuk menghasilkan representasi terdistribusi dari kata-kata:

 - **Continuous bag-of-words** (CBoW) — dalam arsitektur ini, kita melatih model untuk memprediksi sebuah kata berdasarkan konteks di sekitarnya. Diberikan ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, tujuan model adalah memprediksi $W_0$ dari $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** adalah kebalikan dari CBoW. Model menggunakan jendela kata-kata konteks di sekitar untuk memprediksi kata saat ini.

CBoW lebih cepat, sedangkan skip-gram lebih lambat, tetapi lebih baik dalam merepresentasikan kata-kata yang jarang muncul.

![Gambar menunjukkan algoritma CBoW dan Skip-Gram untuk mengonversi kata-kata menjadi vektor.](../../../../../translated_images/id/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Untuk bereksperimen dengan embedding word2vec yang telah dilatih sebelumnya pada dataset Google News, kita dapat menggunakan pustaka **gensim**. Di bawah ini kita menemukan kata-kata yang paling mirip dengan 'neural'

> **Note:** Saat pertama kali membuat vektor kata, mengunduhnya mungkin memerlukan waktu!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Kita juga dapat menghitung penyematan vektor dari kata, untuk digunakan dalam pelatihan model klasifikasi (kami hanya menunjukkan 20 komponen pertama dari vektor untuk kejelasan):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Hal hebat tentang penyematan semantik adalah bahwa Anda dapat memanipulasi pengkodean vektor untuk mengubah semantik. Sebagai contoh, kita dapat meminta untuk menemukan sebuah kata, yang representasi vektornya sedekat mungkin dengan kata *raja* dan *wanita*, serta sejauh mungkin dari kata *pria*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Baik CBoW maupun Skip-Grams adalah embedding "prediktif", karena hanya mempertimbangkan konteks lokal. Word2Vec tidak memanfaatkan konteks global.

**FastText** mengembangkan Word2Vec dengan mempelajari representasi vektor untuk setiap kata dan n-gram karakter yang ditemukan dalam setiap kata. Nilai dari representasi tersebut kemudian dirata-rata menjadi satu vektor pada setiap langkah pelatihan. Meskipun ini menambah banyak perhitungan tambahan pada pra-pelatihan, metode ini memungkinkan embedding kata untuk menyandikan informasi sub-kata.

Metode lain, **GloVe**, memanfaatkan ide matriks ko-occurence, menggunakan metode neural untuk mendekomposisi matriks ko-occurence menjadi vektor kata yang lebih ekspresif dan non-linear.

Anda dapat mencoba contoh ini dengan mengganti embedding ke FastText dan GloVe, karena gensim mendukung beberapa model embedding kata yang berbeda.


## Menggunakan Embedding yang Sudah Dilatih di PyTorch

Kita dapat memodifikasi contoh di atas untuk mengisi matriks dalam lapisan embedding kita dengan embedding semantis, seperti Word2Vec. Kita perlu mempertimbangkan bahwa kosakata dari embedding yang sudah dilatih dan korpus teks kita kemungkinan besar tidak akan cocok, sehingga kita akan menginisialisasi bobot untuk kata-kata yang hilang dengan nilai acak:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


Sekarang mari kita latih model kita. Perhatikan bahwa waktu yang diperlukan untuk melatih model jauh lebih lama dibandingkan dengan contoh sebelumnya, karena ukuran lapisan embedding yang lebih besar, dan dengan demikian jumlah parameter yang jauh lebih tinggi. Selain itu, karena hal ini, kita mungkin perlu melatih model kita pada lebih banyak contoh jika kita ingin menghindari overfitting.


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Dalam kasus kami, kami tidak melihat peningkatan akurasi yang signifikan, kemungkinan disebabkan oleh perbedaan kosakata yang cukup besar.  
Untuk mengatasi masalah perbedaan kosakata, kita dapat menggunakan salah satu solusi berikut:  
* Melatih ulang model word2vec pada kosakata kita  
* Memuat dataset kita dengan kosakata dari model word2vec yang sudah dilatih sebelumnya. Kosakata yang digunakan untuk memuat dataset dapat ditentukan selama proses pemuatan.  

Pendekatan yang terakhir tampaknya lebih mudah, terutama karena kerangka kerja PyTorch `torchtext` memiliki dukungan bawaan untuk embeddings. Sebagai contoh, kita dapat membuat kosakata berbasis GloVe dengan cara berikut:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Kosakata yang dimuat memiliki operasi dasar berikut:
* Kamus `vocab.stoi` memungkinkan kita mengubah kata menjadi indeks dalam kamusnya
* `vocab.itos` melakukan kebalikan - mengubah angka menjadi kata
* `vocab.vectors` adalah array vektor embedding, jadi untuk mendapatkan embedding dari sebuah kata `s` kita perlu menggunakan `vocab.vectors[vocab.stoi[s]]`

Berikut adalah contoh manipulasi embedding untuk menunjukkan persamaan **kind-man+woman = queen** (saya harus menyesuaikan koefisien sedikit agar berhasil):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Untuk melatih classifier menggunakan embedding tersebut, kita pertama-tama perlu mengenkripsi dataset kita menggunakan kosakata GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Seperti yang telah kita lihat di atas, semua vektor embedding disimpan dalam matriks `vocab.vectors`. Hal ini membuatnya sangat mudah untuk memuat bobot tersebut ke dalam bobot lapisan embedding dengan penyalinan sederhana:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

Sekarang mari kita latih model kita dan lihat apakah kita mendapatkan hasil yang lebih baik:


In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Salah satu alasan mengapa kita tidak melihat peningkatan akurasi yang signifikan adalah karena beberapa kata dari dataset kita tidak ada dalam kosakata GloVe yang telah dilatih sebelumnya, sehingga kata-kata tersebut pada dasarnya diabaikan. Untuk mengatasi hal ini, kita dapat melatih embedding kita sendiri pada dataset kita.


## Embedding Kontekstual

Salah satu keterbatasan utama dari representasi embedding pra-latih tradisional seperti Word2Vec adalah masalah disambiguasi makna kata. Meskipun embedding pra-latih dapat menangkap sebagian makna kata dalam konteks, semua kemungkinan makna dari sebuah kata dikodekan ke dalam embedding yang sama. Hal ini dapat menyebabkan masalah pada model hilir, karena banyak kata seperti kata 'play' memiliki makna yang berbeda tergantung pada konteks penggunaannya.

Sebagai contoh, kata 'play' dalam dua kalimat berikut memiliki makna yang cukup berbeda:
- Saya pergi menonton sebuah **play** di teater.
- John ingin **play** dengan teman-temannya.

Embedding pra-latih di atas merepresentasikan kedua makna kata 'play' tersebut dalam embedding yang sama. Untuk mengatasi keterbatasan ini, kita perlu membangun embedding berdasarkan **model bahasa**, yang dilatih pada korpus teks yang besar, dan *memahami* bagaimana kata-kata dapat disusun dalam berbagai konteks. Membahas embedding kontekstual berada di luar cakupan tutorial ini, tetapi kita akan kembali membahasnya saat berbicara tentang model bahasa di unit berikutnya.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
